In [1]:
import pandas as pd;
import os;
import pathlib;
import re
import numpy as np

In [2]:
base = pd.read_csv('base_api.csv')
Municipios = pd.read_csv('municipios.csv')

In [3]:
base

,Unnamed: 0,id,name,publicLocation,location,shortDescription,longDescription,status,emailPublico,telefonePublico,...,terms,emailPrivado,telefone1,telefone2,En_Pais,nomeCompleto,documento,raca,dataDeNascimento,genero
0,0,1,Secretaria de Cidadania e Diversidade Cultural,1.0,"{'latitude': '-15.7950464', 'longitude': '-47....",NaN,NaN,1,culturaviva@cultura.gov.br,(61) 2024-2234,...,"{'tag': [], 'area': ['Gestão Cultural']}",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,2,Marcos Lima,1.0,"{'latitude': '-22.5797501', 'longitude': '-47....",NaN,NaN,1,limamrl@hotmail.com,NaN,...,"{'tag': [], 'area': ['Gestão Cultural', 'Músic...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,3,Sem Nome,1.0,"{'latitude': '0', 'longitude': '0'}",NaN,NaN,1,NaN,NaN,...,"{'tag': [], 'area': []}",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,4,NaN,1.0,"{'latitude': '0', 'longitude': '0'}",NaN,NaN,1,NaN,NaN,...,"{'tag': [], 'area': []}",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,5,NaN,1.0,"{'latitude': '0', 'longitude': '0'}",NaN,NaN,1,NaN,NaN,...,"{'tag': [], 'area': []}",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
159416,159416,302042,NaN,1.0,"{'latitude': '0', 'longitude': '0'}",NaN,NaN,1,NaN,NaN,...,"{'tag': [], 'area': []}",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
159417,159417,302043,NaN,0.0,"{'latitude': 0, 'longitude': 0}",NaN,NaN,1,NaN,NaN,...,"{'tag': [], 'area': []}",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
159418,159418,302046,Eliezer A. de Oliveira,0.0,"{'latitude': 0, 'longitude': 0}",NaN,NaN,1,eliezerado@hotmail.com,(49) 99173-6668,...,"{'tag': [], 'area': ['Educação', 'Literatura']}",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
159419,159419,302047,NaN,1.0,"{'latitude': '0', 'longitude': '0'}",NaN,NaN,1,NaN,NaN,...,"{'tag': [], 'area': []}",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Tratamento dos dados

In [5]:
#Excluindo colunas vazias
colunas_para_excluir = ['emailPrivado', 'telefone1', 'telefone2', 'En_Pais', 'nomeCompleto', 'documento', 'raca', 'dataDeNascimento', 'genero','Unnamed: 0']
base = base.drop(columns=colunas_para_excluir)

In [6]:
#Renomeando colunas
novos_nomes = {
    'name': 'Nome',
    'publicLocation': 'Localização_pública',
    'location': 'Geolocalização',
    'shortDescription': 'Descrição_curta',
    'longDescription': 'Descrição_longa',
    'status': 'Status',
    'emailPublico': 'E-mail',
    'telefonePublico': 'Telefone',
    'endereco': 'Endereço',
    'En_Municipio': 'Município',
    'En_Estado': 'UF',
    'geoEstado': 'Estado',
    'geoMesorregiao': 'Mesorregião',
    'terms': 'Termos'
}

base = base.rename(columns=novos_nomes)

In [7]:
#Excluindo linhas vazias (sem informações)
base['Nome'] = base['Nome'].replace(r'^\s*$', None, regex=True)
base['E-mail'] = base['E-mail'].replace(r'^\s*$', None, regex=True)
base['Telefone'] = base['Telefone'].replace(r'^\s*$', None, regex=True)

base['Nome'] = base['Nome'].replace('', None)
base['E-mail'] = base['E-mail'].replace('', None)
base['Telefone'] = base['Telefone'].replace('', None)

# Substituir "Sem Nome" por None
base['Nome'] = base['Nome'].replace('Sem Nome', None)

base = base.dropna(subset=['Nome', 'E-mail', 'Telefone'], how='all')
base

,id,Nome,Localização_pública,Geolocalização,Descrição_curta,Descrição_longa,Status,E-mail,Telefone,Endereço,Município,UF,Estado,Mesorregião,Termos
0,1,Secretaria de Cidadania e Diversidade Cultural,1.0,"{'latitude': '-15.7950464', 'longitude': '-47....",NaN,NaN,1,culturaviva@cultura.gov.br,(61) 2024-2234,"SCS Quadra 9, SN, Ed. Parque Cidade Corporate ...",Brasília,DF,DISTRITO FEDERAL,DISTRITO FEDERAL,"{'tag': [], 'area': ['Gestão Cultural']}"
1,2,Marcos Lima,1.0,"{'latitude': '-22.5797501', 'longitude': '-47....",NaN,NaN,1,limamrl@hotmail.com,NaN,"Rua Ester Scartezine, 26 , Olga Veroni, 13487-...",Limeira,SP,SÃO PAULO,PIRACICABA,"{'tag': [], 'area': ['Gestão Cultural', 'Músic..."
5,6,Branca,0.0,"{'latitude': 0, 'longitude': 0}",SCHULZ,NaN,1,NaN,NaN,NaN,NaN,NaN,DISTRITO FEDERAL,DISTRITO FEDERAL,"{'tag': [], 'area': ['Teatro']}"
6,7,SCDC,1.0,"{'latitude': '-28.3885333', 'longitude': '-53....",bdsubfduifbdofdiofsd,kkkkkkkkkkkkkkkk,1,NaN,NaN,NaN,Ijuí,RS,RIO GRANDE DO SUL,NOROESTE RIO-GRANDENSE,"{'tag': [], 'area': ['Antropologia', 'Outros']}"
7,8,Plataforma Rede Cultura Viva,1.0,"{'latitude': '0', 'longitude': '0'}",NaN,NaN,1,NaN,NaN,NaN,Ijui,RS,NaN,NaN,"{'tag': [], 'area': []}"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
159410,302034,Centro de Estudo Pesquisa e Intervenção Ribeir...,0.0,"{'latitude': 0, 'longitude': 0}",NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"{'tag': [], 'area': []}"
159412,302036,ABS,0.0,"{'latitude': 0, 'longitude': 0}",NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"{'tag': [], 'area': []}"
159414,302039,CEPI-NEVES,0.0,"{'latitude': 0, 'longitude': 0}",NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"{'tag': [], 'area': []}"
159415,302041,Aryadne Barbosa,0.0,"{'latitude': 0, 'longitude': 0}",NaN,NaN,1,aryadnebarbosa2005@gmail.com,NaN,NaN,NaN,NaN,NaN,NaN,"{'tag': [], 'area': ['Outros']}"


In [8]:
def is_email(text):
    # Expressão regular para identificar e-mails
    email_regex = r'^[a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+$'
    if isinstance(text, str) and re.match(email_regex, text):
        return True
    return False

# Aplicar a função de ajuste
def ajustar_nome_email(row):
    if is_email(row['Nome']):
        # Separar a parte do nome (antes do @) e o e-mail completo
        nome_partes = row['Nome'].split('@')
        row['E-mail'] = row['Nome']  # Mover o e-mail para a coluna E-mail
        row['Nome'] = nome_partes[0]  # Manter apenas a parte antes do @ como Nome
    return row

# Aplicar a função a cada linha do DataFrame
base = base.apply(ajustar_nome_email, axis=1)

# Exibir o DataFrame após as modificações
print(base)

            id                                               Nome  \
0            1     Secretaria de Cidadania e Diversidade Cultural   
1            2                                        Marcos Lima   
5            6                                             Branca   
6            7                                               SCDC   
7            8                       Plataforma Rede Cultura Viva   
...        ...                                                ...   
159410  302034  Centro de Estudo Pesquisa e Intervenção Ribeir...   
159412  302036                                                ABS   
159414  302039                                         CEPI-NEVES   
159415  302041                                    Aryadne Barbosa   
159418  302046                             Eliezer A. de Oliveira   

        Localização_pública  \
0                       1.0   
1                       1.0   
5                       0.0   
6                       1.0   
7               

In [9]:
# Substituindo os valores na coluna 'Localização pública'
base['Localização_pública'] = base['Localização_pública'].replace({1.0: 'SIM', 0.0: 'NÃO'})

In [10]:
# Dicionário de mapeamento de UF para Estado e vice-versa
uf_to_estado = {
    'AC': 'ACRE', 'AL': 'ALAGOAS', 'AM': 'AMAZONAS', 'AP': 'AMAPÁ', 'BA': 'BAHIA', 'CE': 'CEARÁ', 'DF': 'DISTRITO FEDERAL', 'ES': 'ESPÍRITO SANTO',
    'GO': 'GOIÁS', 'MA': 'MARANHÃO', 'MT': 'MATO GROSSO', 'MS': 'MATO GROSSO DO SUL', 'MG': 'MINAS GERAIS', 'PA': 'PARÁ', 'PB': 'PARAÍBA', 
    'PR': 'PARANÁ', 'PE': 'PERNAMBUCO', 'PI': 'PIAUÍ', 'RJ': 'RIO DE JANEIRO', 'RN': 'RIO GRANDE DO NORTE', 'RS': 'RIO GRANDE DO SUL', 
    'RO': 'RONDÔNIA', 'RR': 'RORAIMA', 'SC': 'SANTA CATARINA', 'SP': 'SÃO PAULO', 'SE': 'SERGIPE', 'TO': 'TOCANTINS'
}
estado_to_uf = {v: k for k, v in uf_to_estado.items()}

# Função para preencher os valores faltantes
def preencher_valores(row):
    if pd.isna(row['UF']) and pd.notna(row['Estado']):
        row['UF'] = estado_to_uf.get(row['Estado'], row['UF'])
    if pd.isna(row['Estado']) and pd.notna(row['UF']):
        row['Estado'] = uf_to_estado.get(row['UF'], row['Estado'])
    return row

base = base.apply(preencher_valores, axis=1)

In [11]:
mapeamento_uf = {
    'Ceará': 'CE',
    'GOIÁS': 'GO',
    'Viena': 'Exterior',
    'Catalunya': 'Exterior',
    'Bélgica': 'Exterior',
    'Bruxelas': 'Exterior',
    'Illinois': 'Exterior',
    'Barcelona': 'Exterior',
    'rio de janeiro': 'RJ',
    'Rio de Janeiro': 'RJ',
    'ESPÍRITO SANTO': 'ES',
    'SANTA CATARINA': 'SC',
    'Sverdlovsk': 'Exterior',
    'Rio Grande do Norte': 'RN',
    'New York': 'Exterior',
    'são paulo': 'SP',
    'df': 'DF',
    'bruxelles': 'Exterior',
    'Paraíba': 'PB',
    'Massachusetts': 'Exterior',
    'Mato Grosso': 'MT',
    'Santa Catarina': 'SC',
    'MINAS GERAIS': 'MG',
    'Bahia': 'BA',
    'Rio Grande do Sul': 'RS',
    'Pará': 'PA',
    'RIO DE JANEIRO': 'RJ',
    'SÃO PAULO': 'SP',
    'Luanda': 'Exterior',
    'GOIAS': 'GO',
    'São Paulo': 'SP',
    'TERRASSA': 'Exterior',
    'Santarém': 'PA',
    'Amazonas': 'AM',
    'nan': np.nan
}

# Substituindo os valores na coluna 'UF' e tratando "nan" como np.nan
base['UF'] = base['UF'].replace(mapeamento_uf)

# Convertendo strings "nan" explícitas para np.nan
base['UF'] = base['UF'].replace('nan', np.nan)

In [12]:
valores_distintos_uf = base['UF'].unique()

# Convertendo para lista
lista_valores_distintos_uf = list(valores_distintos_uf)

In [13]:
import ast
# Função para limpar e separar os termos e áreas
def process_termos(row):
    try:
        termos_dict = ast.literal_eval(row)
        tags = ', '.join(termos_dict.get('tag', []))
        areas = ', '.join(termos_dict.get('area', []))
        return tags, areas
    except (ValueError, SyntaxError):
        return '', ''

# Aplicando a função ao DataFrame usando o método apply
base[['Termos_Limpos', 'Área']] = base['Termos'].apply(lambda x: pd.Series(process_termos(x)))

base

,id,Nome,Localização_pública,Geolocalização,Descrição_curta,Descrição_longa,Status,E-mail,Telefone,Endereço,Município,UF,Estado,Mesorregião,Termos,Termos_Limpos,Área
0,1,Secretaria de Cidadania e Diversidade Cultural,SIM,"{'latitude': '-15.7950464', 'longitude': '-47....",NaN,NaN,1,culturaviva@cultura.gov.br,(61) 2024-2234,"SCS Quadra 9, SN, Ed. Parque Cidade Corporate ...",Brasília,DF,DISTRITO FEDERAL,DISTRITO FEDERAL,"{'tag': [], 'area': ['Gestão Cultural']}",,Gestão Cultural
1,2,Marcos Lima,SIM,"{'latitude': '-22.5797501', 'longitude': '-47....",NaN,NaN,1,limamrl@hotmail.com,NaN,"Rua Ester Scartezine, 26 , Olga Veroni, 13487-...",Limeira,SP,SÃO PAULO,PIRACICABA,"{'tag': [], 'area': ['Gestão Cultural', 'Músic...",,"Gestão Cultural, Música, Artesanato, Outros"
5,6,Branca,NÃO,"{'latitude': 0, 'longitude': 0}",SCHULZ,NaN,1,NaN,NaN,NaN,NaN,DF,DISTRITO FEDERAL,DISTRITO FEDERAL,"{'tag': [], 'area': ['Teatro']}",,Teatro
6,7,SCDC,SIM,"{'latitude': '-28.3885333', 'longitude': '-53....",bdsubfduifbdofdiofsd,kkkkkkkkkkkkkkkk,1,NaN,NaN,NaN,Ijuí,RS,RIO GRANDE DO SUL,NOROESTE RIO-GRANDENSE,"{'tag': [], 'area': ['Antropologia', 'Outros']}",,"Antropologia, Outros"
7,8,Plataforma Rede Cultura Viva,SIM,"{'latitude': '0', 'longitude': '0'}",NaN,NaN,1,NaN,NaN,NaN,Ijui,RS,RIO GRANDE DO SUL,NaN,"{'tag': [], 'area': []}",,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
159410,302034,Centro de Estudo Pesquisa e Intervenção Ribeir...,NÃO,"{'latitude': 0, 'longitude': 0}",NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"{'tag': [], 'area': []}",,
159412,302036,ABS,NÃO,"{'latitude': 0, 'longitude': 0}",NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"{'tag': [], 'area': []}",,
159414,302039,CEPI-NEVES,NÃO,"{'latitude': 0, 'longitude': 0}",NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"{'tag': [], 'area': []}",,
159415,302041,Aryadne Barbosa,NÃO,"{'latitude': 0, 'longitude': 0}",NaN,NaN,1,aryadnebarbosa2005@gmail.com,NaN,NaN,NaN,NaN,NaN,NaN,"{'tag': [], 'area': ['Outros']}",,Outros


In [14]:
# Dicionário de mapeamento de estados
mapeamento_estados = {
    'AC': 'ACRE',
    'AL': 'ALAGOAS',
    'AP': 'AMAPA',
    'AM': 'AMAZONAS',
    'BA': 'BAHIA',
    'CE': 'CEARA',
    'DF': 'DISTRITO FEDERAL',
    'ES': 'ESPIRITO SANTO',
    'GO': 'GOIAS',
    'MA': 'MARANHAO',
    'MT': 'MATO GROSSO',
    'MS': 'MATO GROSSO DO SUL',
    'MG': 'MINAS GERAIS',
    'PA': 'PARA',
    'PB': 'PARAIBA',
    'PR': 'PARANA',
    'PE': 'PERNAMBUCO',
    'PI': 'PIAUI',
    'RJ': 'RIO DE JANEIRO',
    'RN': 'RIO GRANDE DO NORTE',
    'RS': 'RIO GRANDE DO SUL',
    'RO': 'RONDONIA',
    'RR': 'RORAIMA',
    'SC': 'SANTA CATARINA',
    'SP': 'SAO PAULO',
    'SE': 'SERGIPE',
    'TO': 'TOCANTINS',
    'Exterior': 'EXTERIOR',
}

# Criando a nova coluna 'Estado_mapa' baseada na coluna 'UF'
base['Estado_mapa'] = base['UF'].map(mapeamento_estados)
base

,id,Nome,Localização_pública,Geolocalização,Descrição_curta,Descrição_longa,Status,E-mail,Telefone,Endereço,Município,UF,Estado,Mesorregião,Termos,Termos_Limpos,Área,Estado_mapa
0,1,Secretaria de Cidadania e Diversidade Cultural,SIM,"{'latitude': '-15.7950464', 'longitude': '-47....",NaN,NaN,1,culturaviva@cultura.gov.br,(61) 2024-2234,"SCS Quadra 9, SN, Ed. Parque Cidade Corporate ...",Brasília,DF,DISTRITO FEDERAL,DISTRITO FEDERAL,"{'tag': [], 'area': ['Gestão Cultural']}",,Gestão Cultural,DISTRITO FEDERAL
1,2,Marcos Lima,SIM,"{'latitude': '-22.5797501', 'longitude': '-47....",NaN,NaN,1,limamrl@hotmail.com,NaN,"Rua Ester Scartezine, 26 , Olga Veroni, 13487-...",Limeira,SP,SÃO PAULO,PIRACICABA,"{'tag': [], 'area': ['Gestão Cultural', 'Músic...",,"Gestão Cultural, Música, Artesanato, Outros",SAO PAULO
5,6,Branca,NÃO,"{'latitude': 0, 'longitude': 0}",SCHULZ,NaN,1,NaN,NaN,NaN,NaN,DF,DISTRITO FEDERAL,DISTRITO FEDERAL,"{'tag': [], 'area': ['Teatro']}",,Teatro,DISTRITO FEDERAL
6,7,SCDC,SIM,"{'latitude': '-28.3885333', 'longitude': '-53....",bdsubfduifbdofdiofsd,kkkkkkkkkkkkkkkk,1,NaN,NaN,NaN,Ijuí,RS,RIO GRANDE DO SUL,NOROESTE RIO-GRANDENSE,"{'tag': [], 'area': ['Antropologia', 'Outros']}",,"Antropologia, Outros",RIO GRANDE DO SUL
7,8,Plataforma Rede Cultura Viva,SIM,"{'latitude': '0', 'longitude': '0'}",NaN,NaN,1,NaN,NaN,NaN,Ijui,RS,RIO GRANDE DO SUL,NaN,"{'tag': [], 'area': []}",,,RIO GRANDE DO SUL
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
159410,302034,Centro de Estudo Pesquisa e Intervenção Ribeir...,NÃO,"{'latitude': 0, 'longitude': 0}",NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"{'tag': [], 'area': []}",,,NaN
159412,302036,ABS,NÃO,"{'latitude': 0, 'longitude': 0}",NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"{'tag': [], 'area': []}",,,NaN
159414,302039,CEPI-NEVES,NÃO,"{'latitude': 0, 'longitude': 0}",NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"{'tag': [], 'area': []}",,,NaN
159415,302041,Aryadne Barbosa,NÃO,"{'latitude': 0, 'longitude': 0}",NaN,NaN,1,aryadnebarbosa2005@gmail.com,NaN,NaN,NaN,NaN,NaN,NaN,"{'tag': [], 'area': ['Outros']}",,Outros,NaN


In [15]:

# 1. Criar uma cópia do DataFrame original para comparação
base_original = base.copy()

# 2. Remover espaços em branco dos e-mails
base['E-mail'] = base['E-mail'].str.replace(' ', '')

# 3. Identificar e-mails duplicados
duplicated_emails = base['E-mail'].duplicated(keep=False)  # Identifica duplicados

# 4. Função para ajustar e-mails duplicados adicionando o 'id'
def ajustar_email_duplicado(row):
    if isinstance(row['E-mail'], str) and duplicated_emails[row.name]:  # Verifica se o e-mail é string e duplicado
        email_partes = row['E-mail'].split('@')
        if len(email_partes) == 2:  # Verifica se o e-mail tem o formato correto
            email_ajustado = f"{email_partes[0]}+{row['id']}@{email_partes[1]}"
            return email_ajustado
    return row['E-mail']  # Retorna o e-mail original se não for string ou se não for duplicado

# 5. Aplicar a função de ajuste apenas nos e-mails duplicados
base['E-mail'] = base.apply(ajustar_email_duplicado, axis=1)


In [16]:
import unidecode
from rapidfuzz import process, fuzz

# Função para normalizar os nomes dos municípios (somente para criar a chave de comparação)
def normalizar_nome(nome):
    if isinstance(nome, str):  # Verifica se o valor é uma string
        # Remover caracteres indesejados como barras, pipes, etc.
        nome = unidecode.unidecode(nome)  # Remove acentos
        nome = nome.lower()  # Converte para minúsculas
        nome = nome.replace('|', ' ')  # Substitui pipes por espaço
        nome = nome.replace('-', ' ')  # Substitui traços por espaço
        nome = nome.replace('\\', ' ')  # Remove barras invertidas
        nome = nome.split('(')[0].split('/')[0].strip()  # Remove siglas e conteúdo entre barras e parênteses
    else:
        nome = ''  # Caso não seja string, retorna uma string vazia
    return nome

# 1. Criar uma coluna normalizada na sua base para ser usada como chave de comparação
base['Municipio_Normalizado'] = base['Município'].apply(normalizar_nome)

# 2. Criar uma coluna normalizada na base de municípios oficial para ser usada como chave de comparação
Municipios['Municipio_Normalizado'] = Municipios['NOME'].apply(normalizar_nome)

# 3. Fazer o merge usando a coluna normalizada como chave, mas manter a coluna 'NOME' da base oficial
base_corrigida = pd.merge(base, Municipios[['NOME', 'Municipio_Normalizado']], 
                          on='Municipio_Normalizado', how='left')

# 4. Preservar valores vazios (caso o município original esteja em branco)
base_corrigida['NOME'] = base_corrigida.apply(
    lambda row: '' if pd.isna(row['Município']) or row['Município'] == '' else row['NOME'],
    axis=1
)

# 5. Caso não seja possível corrigir, preencher com 'Município não identificado'
base_corrigida['NOME'] = base_corrigida['NOME'].fillna('Município não identificado')

# 6. Remover a coluna 'Municipio_Normalizado', se não for mais necessária
base_corrigida = base_corrigida.drop(columns=['Municipio_Normalizado'])

# 7. Exibir os resultados, mostrando o nome original e o nome corrigido
print(base_corrigida[['Município', 'NOME']])

base = base_corrigida

base = base.drop(columns=['Município'])
base = base.rename(columns={'NOME':'Município'})
                  
base

       Município      NOME
0       Brasília  Brasília
1        Limeira   Limeira
2            NaN          
3           Ijuí      Ijuí
4           Ijui      Ijuí
...          ...       ...
146749       NaN          
146750       NaN          
146751       NaN          
146752       NaN          
146753       NaN          

[146754 rows x 2 columns]


,id,Nome,Localização_pública,Geolocalização,Descrição_curta,Descrição_longa,Status,E-mail,Telefone,Endereço,UF,Estado,Mesorregião,Termos,Termos_Limpos,Área,Estado_mapa,Município
0,1,Secretaria de Cidadania e Diversidade Cultural,SIM,"{'latitude': '-15.7950464', 'longitude': '-47....",NaN,NaN,1,culturaviva@cultura.gov.br,(61) 2024-2234,"SCS Quadra 9, SN, Ed. Parque Cidade Corporate ...",DF,DISTRITO FEDERAL,DISTRITO FEDERAL,"{'tag': [], 'area': ['Gestão Cultural']}",,Gestão Cultural,DISTRITO FEDERAL,Brasília
1,2,Marcos Lima,SIM,"{'latitude': '-22.5797501', 'longitude': '-47....",NaN,NaN,1,limamrl+2@hotmail.com,NaN,"Rua Ester Scartezine, 26 , Olga Veroni, 13487-...",SP,SÃO PAULO,PIRACICABA,"{'tag': [], 'area': ['Gestão Cultural', 'Músic...",,"Gestão Cultural, Música, Artesanato, Outros",SAO PAULO,Limeira
2,6,Branca,NÃO,"{'latitude': 0, 'longitude': 0}",SCHULZ,NaN,1,NaN,NaN,NaN,DF,DISTRITO FEDERAL,DISTRITO FEDERAL,"{'tag': [], 'area': ['Teatro']}",,Teatro,DISTRITO FEDERAL,
3,7,SCDC,SIM,"{'latitude': '-28.3885333', 'longitude': '-53....",bdsubfduifbdofdiofsd,kkkkkkkkkkkkkkkk,1,NaN,NaN,NaN,RS,RIO GRANDE DO SUL,NOROESTE RIO-GRANDENSE,"{'tag': [], 'area': ['Antropologia', 'Outros']}",,"Antropologia, Outros",RIO GRANDE DO SUL,Ijuí
4,8,Plataforma Rede Cultura Viva,SIM,"{'latitude': '0', 'longitude': '0'}",NaN,NaN,1,NaN,NaN,NaN,RS,RIO GRANDE DO SUL,NaN,"{'tag': [], 'area': []}",,,RIO GRANDE DO SUL,Ijuí
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
146749,302034,Centro de Estudo Pesquisa e Intervenção Ribeir...,NÃO,"{'latitude': 0, 'longitude': 0}",NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,"{'tag': [], 'area': []}",,,NaN,
146750,302036,ABS,NÃO,"{'latitude': 0, 'longitude': 0}",NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,"{'tag': [], 'area': []}",,,NaN,
146751,302039,CEPI-NEVES,NÃO,"{'latitude': 0, 'longitude': 0}",NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,"{'tag': [], 'area': []}",,,NaN,
146752,302041,Aryadne Barbosa,NÃO,"{'latitude': 0, 'longitude': 0}",NaN,NaN,1,aryadnebarbosa2005@gmail.com,NaN,NaN,NaN,NaN,NaN,"{'tag': [], 'area': ['Outros']}",,Outros,NaN,


In [17]:
base = base.drop_duplicates(subset=['id'], keep='first')

base

,id,Nome,Localização_pública,Geolocalização,Descrição_curta,Descrição_longa,Status,E-mail,Telefone,Endereço,UF,Estado,Mesorregião,Termos,Termos_Limpos,Área,Estado_mapa,Município
0,1,Secretaria de Cidadania e Diversidade Cultural,SIM,"{'latitude': '-15.7950464', 'longitude': '-47....",NaN,NaN,1,culturaviva@cultura.gov.br,(61) 2024-2234,"SCS Quadra 9, SN, Ed. Parque Cidade Corporate ...",DF,DISTRITO FEDERAL,DISTRITO FEDERAL,"{'tag': [], 'area': ['Gestão Cultural']}",,Gestão Cultural,DISTRITO FEDERAL,Brasília
1,2,Marcos Lima,SIM,"{'latitude': '-22.5797501', 'longitude': '-47....",NaN,NaN,1,limamrl+2@hotmail.com,NaN,"Rua Ester Scartezine, 26 , Olga Veroni, 13487-...",SP,SÃO PAULO,PIRACICABA,"{'tag': [], 'area': ['Gestão Cultural', 'Músic...",,"Gestão Cultural, Música, Artesanato, Outros",SAO PAULO,Limeira
2,6,Branca,NÃO,"{'latitude': 0, 'longitude': 0}",SCHULZ,NaN,1,NaN,NaN,NaN,DF,DISTRITO FEDERAL,DISTRITO FEDERAL,"{'tag': [], 'area': ['Teatro']}",,Teatro,DISTRITO FEDERAL,
3,7,SCDC,SIM,"{'latitude': '-28.3885333', 'longitude': '-53....",bdsubfduifbdofdiofsd,kkkkkkkkkkkkkkkk,1,NaN,NaN,NaN,RS,RIO GRANDE DO SUL,NOROESTE RIO-GRANDENSE,"{'tag': [], 'area': ['Antropologia', 'Outros']}",,"Antropologia, Outros",RIO GRANDE DO SUL,Ijuí
4,8,Plataforma Rede Cultura Viva,SIM,"{'latitude': '0', 'longitude': '0'}",NaN,NaN,1,NaN,NaN,NaN,RS,RIO GRANDE DO SUL,NaN,"{'tag': [], 'area': []}",,,RIO GRANDE DO SUL,Ijuí
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
146749,302034,Centro de Estudo Pesquisa e Intervenção Ribeir...,NÃO,"{'latitude': 0, 'longitude': 0}",NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,"{'tag': [], 'area': []}",,,NaN,
146750,302036,ABS,NÃO,"{'latitude': 0, 'longitude': 0}",NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,"{'tag': [], 'area': []}",,,NaN,
146751,302039,CEPI-NEVES,NÃO,"{'latitude': 0, 'longitude': 0}",NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,"{'tag': [], 'area': []}",,,NaN,
146752,302041,Aryadne Barbosa,NÃO,"{'latitude': 0, 'longitude': 0}",NaN,NaN,1,aryadnebarbosa2005@gmail.com,NaN,NaN,NaN,NaN,NaN,"{'tag': [], 'area': ['Outros']}",,Outros,NaN,


In [18]:
inativos = pd.read_csv('emails_invalidos.csv', encoding='ISO-8859-1', on_bad_lines='skip')

inativos.rename(columns={'Email': 'E-mail'}, inplace=True)

base['Situação'] = ''

base.loc[base['E-mail'].isin(inativos['E-mail']), 'Situação'] = 'Inativo'

base


C:\Users\DSilva\AppData\Local\Temp\ipykernel_48196\3652854725.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  base['Situação'] = ''


KeyError: 'E-mail'

In [ ]:
base

### Exportação da base

In [ ]:
base.to_csv(r"C:\Users\DSilva\Downloads\MUHDA - PNAB\base_tratada.csv")
#base.to_excel('base_tratada.xlsx', index=False)